# 04 — Phase Transition Detection: Statistical Mechanics of Market Regimes

**Objective:** Apply phase transition framework from statistical mechanics to detect market regime shifts through entropy discontinuities, correlation length divergence (critical slowing down), and observable analogues.

**Venues:** Binance BTCUSDT Perp, Bybit BTCUSDT Perp

**Key question:** Do crypto markets exhibit phase-transition-like behavior during the Jan 30 – Feb 5 crash? Can we detect regime shifts before they fully materialize?

**Golden rule:** Every section ends with *"The trading implication is..."*

## 1. Setup & Data Loading

In [ ]:
import sys
sys.path.insert(0, "..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from src.data import load_processed
from src.entropy import rolling_shannon_entropy
from src.phase_transitions import (
    realised_volatility,
    order_flow_imbalance,
    susceptibility,
    correlation_length,
    detect_entropy_discontinuities,
    classify_regime,
)
from src.visualisation import (
    set_style,
    VENUE_COLOURS,
    REGIME_COLOURS,
    plot_phase_observables,
    plot_correlation_length_timeseries,
    plot_entropy_discontinuities,
    plot_regime_timeline,
)

set_style()

PROCESSED_DIR = Path("../data/processed")
FIGURES_DIR = Path("../figures")
FIGURES_DIR.mkdir(exist_ok=True)

In [ ]:
# Load trade data
binance = load_processed(PROCESSED_DIR / "binance_btcusdt_perp.parquet")
bybit = load_processed(PROCESSED_DIR / "bybit_btcusdt_perp.parquet")
venues = {"binance": binance, "bybit": bybit}

for name, df in venues.items():
    print(f"{name.capitalize()}: {len(df):,} trades | "
          f"{df['timestamp'].min()} → {df['timestamp'].max()}")

In [ ]:
# Reference price series (Binance 1-min last price)
price_1m = (
    binance.set_index("timestamp")["price"]
    .resample("1min")
    .last()
    .dropna()
)
print(f"Price series: {len(price_1m)} 1-min bars, "
      f"${price_1m.min():,.0f} – ${price_1m.max():,.0f}")

## 2. Temperature and Order Parameter Analogues (Task 4a)

*Why this matters for the project: In statistical mechanics, phase transitions are characterized by observable quantities like temperature, magnetization (order parameter), and susceptibility. Here we map market observables to these concepts: volatility serves as temperature (energy of the system), net order flow imbalance serves as magnetization (degree of order), and variance of imbalance serves as susceptibility (sensitivity to perturbations). These mappings allow us to apply the phase transition framework to financial markets.*

In [ ]:
# Focus on Binance (primary venue)
venue_name = "binance"
df = venues[venue_name]

print(f"Computing phase observables for {venue_name.capitalize()}...")

In [ ]:
# Compute temperature (realized volatility) on 1-second price series
price_1s = df.set_index("timestamp")["price"].resample("1s").last().dropna()

# Window = 300 seconds (5 minutes)
temperature = realised_volatility(price_1s, window=300)
temperature = temperature.dropna()

print(f"Temperature (volatility):")
print(f"  Mean: {temperature.mean():.6f}")
print(f"  Std: {temperature.std():.6f}")
print(f"  Min: {temperature.min():.6f}")
print(f"  Max: {temperature.max():.6f}")

In [ ]:
# Compute order parameter (order flow imbalance) on trade signs
signs = df["trade_sign"].values

# Window = 300 trades (~2.6 seconds at 115 trades/s)
imbalance = order_flow_imbalance(signs, window=300)

# Create series with timestamps
imbalance_series = pd.Series(imbalance, index=df["timestamp"])
imbalance_series = imbalance_series.dropna()

print(f"\nOrder parameter (imbalance):")
print(f"  Mean: {imbalance_series.mean():.6f}")
print(f"  Std: {imbalance_series.std():.6f}")
print(f"  Min: {imbalance_series.min():.6f}")
print(f"  Max: {imbalance_series.max():.6f}")

In [ ]:
# Compute susceptibility (variance of imbalance)
# Window = 1000 trades (~8.7 seconds)
susc = susceptibility(imbalance, window=1000)
susc_series = pd.Series(susc, index=df["timestamp"])
susc_series = susc_series.dropna()

print(f"\nSusceptibility:")
print(f"  Mean: {susc_series.mean():.6f}")
print(f"  Std: {susc_series.std():.6f}")
print(f"  Min: {susc_series.min():.6f}")
print(f"  Max: {susc_series.max():.6f}")

In [ ]:
# Resample observables to 1-minute for cleaner visualization
temp_1m = temperature.resample("1min").mean().dropna()
imb_1m = imbalance_series.resample("1min").mean().dropna()
susc_1m = susc_series.resample("1min").mean().dropna()

# Align to common index
common_idx = temp_1m.index.intersection(imb_1m.index).intersection(susc_1m.index)
temp_1m = temp_1m.loc[common_idx]
imb_1m = imb_1m.loc[common_idx]
susc_1m = susc_1m.loc[common_idx]

print(f"\n1-minute resampled observables: {len(common_idx)} data points")

In [ ]:
# Plot all observables
fig = plot_phase_observables(
    timestamps=common_idx,
    temperature=temp_1m.values,
    order_parameter=imb_1m.values,
    susceptibility=susc_1m.values,
    prices=price_1m,
    title=f"{venue_name.capitalize()} — Phase Transition Observables",
)
fig.savefig(FIGURES_DIR / "04_temperature_order_parameter.png", dpi=150, bbox_inches="tight")
plt.show()

**Sanity check:**
- Temperature (volatility) should spike during the Jan 30-31 crash when BTC dropped sharply
- Order parameter should show strong imbalances (near ±1) during directional moves
- Susceptibility should peak during transition periods where the market is most sensitive
- Order parameter mean should be near 0 (market balanced overall) but with significant variation

**The trading implication is:** These thermodynamic analogues provide a unified framework for characterizing market state. High temperature (volatility) with low order parameter (balanced flow) indicates a "hot", chaotic regime where aggressive strategies carry high risk. Low temperature with high order parameter magnitude indicates a "cold", trending regime where directional strategies can be sustained. Susceptibility peaks signal the market is most responsive to new information — ideal conditions for information-based execution strategies but dangerous for pure market-making.

## 3. Correlation Length Analysis (Task 4b)

*Why this matters for the project: Near phase transitions in physics, the correlation length (spatial scale over which system components are correlated) diverges — a phenomenon called "critical slowing down". In markets, we measure correlation length as the timescale over which return autocorrelation decays. If correlation length increases sharply before regime shifts, it provides an early warning signal that the current market state is becoming unstable.*

In [ ]:
# Compute returns on 1-second price series
returns_1s = price_1s.pct_change().dropna().values

print(f"Computing correlation length (this may take several minutes)...")
print(f"Total return observations: {len(returns_1s):,}")

In [ ]:
# Compute rolling correlation length
# Window = 3600 seconds (1 hour), threshold = 1/e
corr_len = correlation_length(
    returns_1s,
    window=3600,
    threshold=1/np.e,
    max_lag=100,
)

# Create time series
corr_len_series = pd.Series(corr_len, index=price_1s.pct_change().dropna().index)
corr_len_series = corr_len_series.dropna()

print(f"\nCorrelation length computed: {len(corr_len_series):,} values")
print(f"  Mean: {corr_len_series.mean():.2f} lags")
print(f"  Std: {corr_len_series.std():.2f} lags")
print(f"  Min: {corr_len_series.min():.2f} lags")
print(f"  Max: {corr_len_series.max():.2f} lags")

In [ ]:
# Resample to 5-minute for visualization
corr_len_5m = corr_len_series.resample("5min").mean().dropna()

print(f"5-minute resampled: {len(corr_len_5m)} data points")

In [ ]:
# Plot correlation length time series
fig = plot_correlation_length_timeseries(
    timestamps=corr_len_5m.index,
    correlation_length=corr_len_5m.values,
    prices=price_1m,
    title=f"{venue_name.capitalize()} — Correlation Length Evolution (Critical Slowing Down)",
)
fig.savefig(FIGURES_DIR / "04_correlation_length.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# Identify correlation length spikes (potential critical points)
threshold_90 = corr_len_5m.quantile(0.90)
spikes = corr_len_5m[corr_len_5m > threshold_90]

print(f"\nCorrelation length spikes (>90th percentile = {threshold_90:.2f} lags):")
print(f"  {len(spikes)} windows detected")
print(f"\nTop 5 spikes:")
for ts, val in spikes.nlargest(5).items():
    price = price_1m.loc[price_1m.index.searchsorted(ts)] if ts in price_1m.index else np.nan
    print(f"  {ts} | corr_len = {val:.2f} lags | price ≈ ${price:,.0f}")

In [ ]:
# Check if correlation length increases precede volatility spikes
# Compute forward volatility (next 30 minutes)
vol_forward = price_1m.pct_change().rolling(30).std().shift(-30)

# Align with correlation length
aligned = pd.DataFrame({
    "corr_len": corr_len_5m,
    "vol_forward": vol_forward.resample("5min").mean(),
}).dropna()

# Correlation between correlation length and forward volatility
corr_coef = aligned["corr_len"].corr(aligned["vol_forward"])
print(f"\nCorrelation between correlation length and 30-min forward volatility: {corr_coef:.4f}")

# Test: high correlation length → high forward volatility?
high_corr_len = aligned[aligned["corr_len"] > threshold_90]
normal_corr_len = aligned[aligned["corr_len"] <= threshold_90]

print(f"\nForward volatility comparison:")
print(f"  High corr_len (>90th pct): mean vol = {high_corr_len['vol_forward'].mean():.6f}")
print(f"  Normal corr_len: mean vol = {normal_corr_len['vol_forward'].mean():.6f}")
print(f"  Ratio: {high_corr_len['vol_forward'].mean() / normal_corr_len['vol_forward'].mean():.2f}x")

**Sanity check:**
- Correlation length should be positive and bounded (between 0 and max_lag)
- Mean correlation length should be in the 1-20 lag range for 1-second returns
- Spikes should occur near known volatility events (Jan 30-31 crash)
- If critical slowing down holds, high correlation length should precede volatility increases

**The trading implication is:** Correlation length serves as an early warning indicator for regime transitions. When correlation length increases sharply (diverges), it signals that the market is approaching a critical point where a phase transition is likely. An HFT desk monitoring correlation length in real-time could detect these precursor signals and adjust risk accordingly: reduce position sizes and widen execution thresholds during critical periods (high correlation length, high susceptibility), then increase aggression once the new regime stabilizes (correlation length normalizes).

## 4. Entropy Discontinuities (Task 4c)

*Why this matters for the project: Phase transitions in physics are classified by their order: first-order transitions show discontinuous jumps in observables (like water freezing), while second-order transitions are continuous but with diverging derivatives (like the Curie point in magnets). By analyzing Shannon entropy from Phase 3 for discontinuities, we can characterize the nature of market regime shifts and correlate them with specific events like liquidation cascades.*

In [ ]:
# Load Shannon entropy from Phase 3 (5-minute windows)
# Recompute here for self-containment
entropy_binance = rolling_shannon_entropy(binance, window="5min")

print(f"Shannon entropy loaded: {len(entropy_binance)} windows")
print(f"  Mean H: {entropy_binance['entropy'].mean():.4f}")
print(f"  Std H: {entropy_binance['entropy'].std():.4f}")

In [ ]:
# Detect entropy discontinuities
entropy_vals = entropy_binance["normalised_entropy"].values
discontinuities, derivative = detect_entropy_discontinuities(
    entropy_vals,
    threshold=2.0,  # 2 standard deviations
)

print(f"\nEntropy discontinuities detected: {discontinuities.sum()}")
print(f"Derivative statistics:")
print(f"  Mean: {derivative.mean():.6f}")
print(f"  Std: {derivative[derivative != 0].std():.6f}")
print(f"  Max |derivative|: {np.abs(derivative).max():.6f}")

In [ ]:
# Plot entropy with discontinuities marked
fig = plot_entropy_discontinuities(
    timestamps=entropy_binance["timestamp"],
    entropy=entropy_vals,
    derivative=derivative,
    discontinuities=discontinuities,
    prices=price_1m,
    title=f"{venue_name.capitalize()} — Entropy Discontinuity Detection",
)
fig.savefig(FIGURES_DIR / "04_entropy_discontinuities.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# Analyze discontinuity events
disc_times = entropy_binance["timestamp"][discontinuities]
disc_entropy = entropy_vals[discontinuities]
disc_deriv = derivative[discontinuities]

print(f"\nTop 10 entropy discontinuities:")
disc_df = pd.DataFrame({
    "timestamp": disc_times.values,
    "entropy": disc_entropy,
    "derivative": disc_deriv,
    "abs_deriv": np.abs(disc_deriv),
})
disc_df = disc_df.sort_values("abs_deriv", ascending=False)

for idx, row in disc_df.head(10).iterrows():
    ts = pd.Timestamp(row["timestamp"])
    price = price_1m.loc[price_1m.index.searchsorted(ts)] if ts in price_1m.index else np.nan
    print(f"  {ts} | H = {row['entropy']:.4f} | dH/dt = {row['derivative']:+.4f} | price ≈ ${price:,.0f}")

In [ ]:
# Correlate discontinuities with price moves
# For each discontinuity, check if it coincides with a large price move
price_changes = price_1m.pct_change(5).abs()  # 5-minute return magnitude

disc_price_moves = []
for ts in disc_times:
    ts = pd.Timestamp(ts)
    if ts in price_changes.index:
        disc_price_moves.append(price_changes.loc[ts])
    else:
        # Find nearest
        idx = price_changes.index.searchsorted(ts)
        if idx < len(price_changes):
            disc_price_moves.append(price_changes.iloc[idx])

if disc_price_moves:
    disc_price_moves = np.array(disc_price_moves)
    mean_all = price_changes.mean()
    mean_disc = disc_price_moves.mean()
    print(f"\nPrice move correlation:")
    print(f"  Mean |return| at discontinuities: {mean_disc:.4%}")
    print(f"  Mean |return| overall: {mean_all:.4%}")
    print(f"  Ratio: {mean_disc / mean_all:.2f}x")

**Sanity check:**
- Entropy discontinuities should align with visible price discontinuities (sharp moves)
- Derivative should be near zero most of the time, with spikes at discontinuities
- The Jan 30-31 crash period should show multiple discontinuities
- Discontinuities should be more frequent during high-volatility periods

**The trading implication is:** Entropy discontinuities mark regime transitions in real-time. Sharp entropy drops (large negative derivative) signal the onset of informed directional trading; sharp entropy increases signal return to randomness/noise. By classifying these as first-order-like (sudden jumps, likely liquidation cascades) versus second-order-like (gradual transitions, likely regime shifts), a desk can tailor its response: first-order transitions require immediate risk reduction, while second-order transitions allow for adaptive repositioning.

## 5. Regime Classification (Task 4d)

*Why this matters for the project: Combining all Phase 4 observables into a unified regime classification provides a complete phase-transition framework for market state. By classifying each time window as "hot" (disordered, high entropy/volatility), "cold" (ordered, low entropy/volatility), or "critical" (transition zone), we can test whether these physics-inspired labels correspond to intuitively different market conditions and whether regime transitions are predictive of future volatility.*

In [ ]:
# Prepare aligned observables at 5-minute resolution
# Resample all to 5min and align
entropy_5m = pd.Series(
    entropy_binance["normalised_entropy"].values,
    index=entropy_binance["timestamp"],
)

vol_5m = temp_1m.resample("5min").mean()
susc_5m = susc_1m.resample("5min").mean()
corr_5m = corr_len_5m

# Align to common index
common = entropy_5m.index.intersection(vol_5m.index).intersection(susc_5m.index).intersection(corr_5m.index)

entropy_aligned = entropy_5m.loc[common].values
vol_aligned = vol_5m.loc[common].values
susc_aligned = susc_5m.loc[common].values
corr_aligned = corr_5m.loc[common].values

print(f"Aligned observables: {len(common)} 5-minute windows")

In [ ]:
# Classify regimes
regimes = classify_regime(
    volatility=vol_aligned,
    entropy=entropy_aligned,
    corr_length=corr_aligned,
    susceptibility=susc_aligned,
)

regime_series = pd.Series(regimes, index=common)

print(f"\nRegime classification:")
for regime in ["hot", "cold", "critical", "transitional", "unknown"]:
    count = (regime_series == regime).sum()
    pct = count / len(regime_series)
    print(f"  {regime.capitalize():15s}: {count:4d} ({pct:.1%})")

In [ ]:
# Plot regime timeline
fig = plot_regime_timeline(
    regimes=regime_series,
    prices=price_1m,
    title=f"{venue_name.capitalize()} — Market Regime Classification",
)
fig.savefig(FIGURES_DIR / "04_regime_classification.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# Regime statistics
regime_stats = []
for regime in ["hot", "cold", "critical", "transitional"]:
    mask = regime_series == regime
    if mask.sum() == 0:
        continue
    regime_stats.append({
        "Regime": regime.capitalize(),
        "Count": mask.sum(),
        "Mean Entropy": entropy_aligned[mask.values].mean(),
        "Mean Volatility": vol_aligned[mask.values].mean(),
        "Mean Corr Length": corr_aligned[mask.values].mean(),
        "Mean Susceptibility": susc_aligned[mask.values].mean(),
    })

regime_stats_df = pd.DataFrame(regime_stats)
print("\nRegime statistics:")
print(regime_stats_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

In [ ]:
# Regime transition matrix
transitions = []
for i in range(1, len(regime_series)):
    curr = regime_series.iloc[i - 1]
    next_regime = regime_series.iloc[i]
    transitions.append((curr, next_regime))

transition_df = pd.DataFrame(transitions, columns=["from", "to"])
transition_matrix = pd.crosstab(
    transition_df["from"],
    transition_df["to"],
    normalize="index",
)

print("\nRegime transition matrix (row = from, col = to):")
print(transition_matrix.to_string(float_format=lambda x: f"{x:.3f}"))

In [ ]:
# Test: are regime changes predictive of forward volatility?
# Identify regime transitions (any change)
regime_changes = (regime_series != regime_series.shift(1))

# Forward volatility (next 30 minutes)
fwd_vol_5m = price_1m.pct_change().rolling(30).std().shift(-30).resample("5min").mean()

# Align
aligned_test = pd.DataFrame({
    "regime_change": regime_changes,
    "fwd_vol": fwd_vol_5m,
}).dropna()

at_transition = aligned_test[aligned_test["regime_change"]]
no_transition = aligned_test[~aligned_test["regime_change"]]

print(f"\nForward volatility after regime transitions:")
print(f"  At transitions: mean vol = {at_transition['fwd_vol'].mean():.6f}")
print(f"  No transitions: mean vol = {no_transition['fwd_vol'].mean():.6f}")
print(f"  Ratio: {at_transition['fwd_vol'].mean() / no_transition['fwd_vol'].mean():.2f}x")

**Sanity check:**
- "Hot" regime should have high entropy and high volatility
- "Cold" regime should have low entropy and low volatility
- "Critical" regime should have high susceptibility or diverging correlation length
- Regime classification should not flip too rapidly (no excessive oscillation)
- The Jan 30-31 crash period should show "critical" or "hot" regimes

**The trading implication is:** The regime classification provides a unified real-time dashboard of market state. When the market enters a "critical" regime (correlation length diverging, susceptibility peaking), it signals an impending regime transition. An HFT desk could reduce position sizes or widen execution thresholds during these unstable periods, and increase aggression once the new regime is established (transition to "hot" or "cold"). The regime transition matrix quantifies persistence and switching probabilities, enabling Markov-chain-based position sizing and execution timing strategies.

## 6. Summary & Preview

In [ ]:
# Summary statistics
print("=" * 80)
print("PHASE TRANSITION ANALYSIS — KEY METRICS")
print("=" * 80)

print(f"\n--- Temperature (Volatility) ---")
print(f"  Mean: {temp_1m.mean():.6f}")
print(f"  Max: {temp_1m.max():.6f} (peak volatility)")

print(f"\n--- Order Parameter (Imbalance) ---")
print(f"  Mean: {imb_1m.mean():.6f}")
print(f"  Range: [{imb_1m.min():.4f}, {imb_1m.max():.4f}]")

print(f"\n--- Correlation Length ---")
print(f"  Mean: {corr_len_5m.mean():.2f} lags")
print(f"  Max: {corr_len_5m.max():.2f} lags (critical slowing down)")
print(f"  Correlation with forward vol: {corr_coef:.4f}")

print(f"\n--- Entropy Discontinuities ---")
print(f"  Total detected: {discontinuities.sum()}")
print(f"  Mean |price move| at discontinuities: {mean_disc:.4%} (vs {mean_all:.4%} overall)")

print(f"\n--- Regime Classification ---")
for regime in ["hot", "cold", "critical", "transitional"]:
    count = (regime_series == regime).sum()
    pct = count / len(regime_series) if len(regime_series) > 0 else 0
    print(f"  {regime.capitalize():15s}: {count} ({pct:.1%})")

print(f"\nForward vol at regime transitions: {at_transition['fwd_vol'].mean() / no_transition['fwd_vol'].mean():.2f}x higher")
print("=" * 80)

### Trading Implications Summary

| Finding | Implication |
|---------|-------------|
| Temperature (volatility) spikes during crash | High temperature = high-risk, chaotic regime; reduce position sizes |
| Order parameter shows strong imbalances during trends | Magnitude of order parameter indicates strength of directional flow |
| Susceptibility peaks at transitions | High susceptibility = market most responsive to new information |
| Correlation length diverges before volatility increases | Early warning signal for regime shifts; adjust risk proactively |
| Entropy discontinuities align with price moves | Sharp entropy changes mark regime transitions in real-time |
| Regime classification quantifies market state | Unified dashboard enables state-dependent execution and position sizing |
| Regime transitions predict higher forward volatility | Transition detection enables predictive risk management |

### What Comes Next

Phase 4 has established a phase transition framework for detecting regime shifts. The observables (temperature, order parameter, susceptibility, correlation length) and regime classification provide a physics-grounded toolkit for characterizing market state.

**Phase 5 (Metastability)** would extend this by identifying quasi-stable price levels where the market lingers before transitioning — analogous to metastable states in physics (like supercooled water). Using free-energy landscape construction and dwell time analysis, we could identify support/resistance levels from first principles and predict breakout probabilities.

**Phase 6 (Synthesis)** will tie everything together: entropy flow (Phase 3) + phase transitions (Phase 4) + metastability (Phase 5) into a unified market state dashboard with concrete trading rules.